Import functions

In [297]:
import requests
import pandas as pd
from urllib import parse
from bs4 import BeautifulSoup
from sqlalchemy import create_engine

Define function: get_source

In [298]:
def get_source(url):
    """Return the source code for the provided URL. 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html. 
    """

    try:
        r = requests.get(url)
        content = r.text
        return content

    except requests.exceptions.RequestException as e:
        print(e)

Define function: get_DOS_opps

In [322]:
def get_DOS_opps(DOS_url, search_term):
    """Return the open opportunities across all pages of the DOS webpage according to a key word search. 

    Args: 
        url (string): URL of the DOS page to scrape.
        search_term (string): key term search for the DOS page. Can be be one or multiple words.

    Returns:
        response (object): HTTP response object from requests_html. 
    """

    opportunities = []
    for page in range(1,1000):
        # create string
        search_term_html = parse.quote_plus(search_term)
        url_full = url + "?statusOpenClosed=open" + "&q=" + search_term_html + "&page=" + str(page) + "" 
        # retreive HTML content 
        content = get_source(url_full)
        # create BeautifulSoup soup object 
        soup = BeautifulSoup(content, "lxml")
        # if no page exists for the page loop, break
        if "Page could not be found" in soup.find("title").text:
            break
        # if no opportunities returned within URL HTML, break 
        elif soup.find("p", class_= "app-search-summary govuk-body-s").span.text == "0":
            break
        # if opportunitues are returned and page exists on page loop, retrieve relevant information 
        else: 
            sections = soup.find_all("li", class_ = "app-search-result")
            for section in sections:
                # retreive opp name
                opp_name = section.h2.a.text
                # print(opp_name)
                # retreive opp url
                opp_url = section.h2.a["href"]
                # print(opp_url)
                # set up dict to store retreived organisation, location, published date, question deadline date and close date
                attributes = []
                for ul in section.find_all("ul"): # always 3 unordered lists
                    if "govuk-list" in ul["class"] and "app-search-result__metadata" in ul["class"] and ul.find("li", class_="govuk-!-display-inline-block") is not None:
                        continue
                    else:
                        for li in ul.find_all("li"):
                            attributes.append(li.text.strip())
                organisation, location, published, questions, close = attributes
                # print(organisation)
                # print(location)
                # print(published)
                # print(questions)
                # print(close)
                # retreive opp description
                description = section.find("p").text.strip()
                # print(description)
                # append all information in opportunties list as a dictionary    
                opportunities.append({"Opportunity Name" : opp_name
                                 ,"Oppurtunity URL" : parse.urlparse(url).scheme + "://" + parse.urlparse(url).netloc + opp_url
                                 ,"Organisation" : organisation.replace("Organisation: ", "")
                                 , "Location" : location.replace("Location: ", "")
                                 , "Published Date" : published.replace("Published: ", "")
                                 , "Questions Deadline Date" : questions.replace("Deadline for asking questions: ", "")
                                 , "Close Date" : close.replace("Closing: ", "")
                                 , "Description" : description})

    return opportunities


testing the function code 

In [255]:
# url = "https://www.digitalmarketplace.service.gov.uk/digital-outcomes-and-specialists/opportunities"
# search_term_html = "data"
# # create string
# search_term_html = urllib.parse.quote_plus(search_term)
# url_full = url + "?statusOpenClosed=open" + "&q=" + search_term_html + "&page=1" 
# # retreive HTML content 
# content = get_source(url_full)
# # create BeautifulSoup soup object 
# soup = BeautifulSoup(content, "lxml")
# # if no page exists for the page loop, break
# if "Page could not be found" in soup.find("title").text:
#     print("Page could not be found")
# # if no opportunities returned within URL HTML, break 
# elif soup.find("p", class_= "app-search-summary govuk-body-s").span.text == "0":
#     print("no opportunities")
# # if opportunitues are returned and page exists on page loop, retrieve relevant information 
# else: 
#     sections = soup.find_all("li", class_ = "app-search-result")
#     for section in sections:
#         opp_name = section.h2.a.text
#         # print(opp_name)
#         opp_url = section.h2.a["href"]
#         # print(opp_url)
#         attributes = []
#         for ul in section.find_all("ul"):
#             for li in ul.find_all("li"):
#                 attributes.append(li.text.strip())
#         organisation, location, name1, name2, published, questions, close = attributes
#         # print(organisation)
#         # print(location)
#         # print(name1)
#         # print(name2)
#         # print(published)
#         # print(questions)
#         # print(close)
#         description = section.find("p").text.strip()
#         # print(description)



In [323]:
opp_list = []
dos_url = "https://www.digitalmarketplace.service.gov.uk/digital-outcomes-and-specialists/opportunities"
search_term_list = ["Guildhall", "DWP", "Digital", "partner"]

for term in search_term_list: 
    opp_list.extend(get_DOS_opps(dos_url, term))

unique_opp_list = []
unique_set = set()

for dict in opp_list:
    if str(dict) not in unique_set:
        unique_opp_list.append(dict)
        unique_set.add(str(dict))
# print(opp_list)
# print(unique_opp_list)


Create datframe and write to csv

In [328]:
df = pd.DataFrame(unique_opp_list)
df.reset_index(drop=True)
df["Remaining Days Until Close"] = pd.to_datetime(df["Close Date"]).dt.date - pd.Timestamp.today().date()
df["Remaining Days Until Close"] = df["Remaining Days Until Close"].apply(lambda x: x.days)
df.sort_values("Remaining Days Until Close", ascending=True) # can pass in list of which columns to sort by, and pass a list in for acsending variable to switch order by column

df.to_csv("C:\\Users\\adams\\Documents\\my_csv.csv")

# Create a database engine
engine = create_engine('sqlite:///test.db')

# Drop the table if it already exists
engine.execute('DROP TABLE IF EXISTS my_table')

# Write the dataframe to a table in the database
df.to_sql('my_table', engine)

# Query the database using SQL syntax
query = 'SELECT * FROM my_table order by [Remaining Days Until Close]'
results = pd.read_sql(query, engine)

# # View the results as a Pandas dataframe
results

,index,Opportunity Name,Oppurtunity URL,Organisation,Location,Published Date,Questions Deadline Date,Close Date,Description,Remaining Days Until Close
0,1,DWP Senior Agile Delivery Manager 6,https://www.digitalmarketplace.service.gov.uk/...,Department for Work and Pensions,North West England,Monday 13 March 2023,Wednesday 15 March 2023,Monday 20 March 2023,Part of an internal team ensuring the deliver...,5
1,2,DWP Senior Agile Delivery Manager 5,https://www.digitalmarketplace.service.gov.uk/...,Department for Work and Pensions,North West England,Monday 13 March 2023,Wednesday 15 March 2023,Monday 20 March 2023,Part of an internal team ensuring the deliver...,5
2,3,DWP Identity Business Analyst 3,https://www.digitalmarketplace.service.gov.uk/...,Department for Works & Pensions,North West England,Monday 13 March 2023,Wednesday 15 March 2023,Monday 20 March 2023,Work with DWP colleagues from across the depar...,5
3,6,"A partner to design, develop, iterate and run ...",https://www.digitalmarketplace.service.gov.uk/...,Northumberland County Council,North East England,Tuesday 7 March 2023,Tuesday 14 March 2023,Tuesday 21 March 2023,"A partner to design, develop, iterate and run ...",6
4,4,Digital Programme Resourcing : OC01 Digital Tr...,https://www.digitalmarketplace.service.gov.uk/...,Oldham Council,North West England,Thursday 9 March 2023,Thursday 16 March 2023,Thursday 23 March 2023,Digital Programme Manager role leads the deliv...,8
5,5,Digital Programme Resourcing : OC02 Digital Bu...,https://www.digitalmarketplace.service.gov.uk/...,Oldham Council,North West England,Thursday 9 March 2023,Thursday 16 March 2023,Thursday 23 March 2023,The Digital BA role will be responsible for un...,8
6,0,Guildhall_School_Firewall_Upgrade_2023,https://www.digitalmarketplace.service.gov.uk/...,Guildhall School of Music and Drama,London,Tuesday 14 March 2023,Tuesday 21 March 2023,Tuesday 28 March 2023,To replace existing on-premise Firewall estate...,13
